In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('horses.csv')

In [3]:
df.head()

,race_number,saddle_number,win_fair_price,win_starting_price,winner
0,1,4,1.7353,1.7098,1
1,1,1,6.0313,6.0914,0
2,1,5,7.6923,7.5101,0
3,1,6,20.3325,20.4978,0
4,1,2,23.9991,23.4710,0


In [4]:
df.tail()

,race_number,saddle_number,win_fair_price,win_starting_price,winner
86643,10000,7,14.4772,13.8269,0
86644,10000,5,29.0062,29.8636,0
86645,10000,3,50.8005,48.9715,0
86646,10000,9,310.1959,305.6448,0
86647,10000,2,621.1930,601.6791,0


In [5]:
df.dtypes

race_number             int64
saddle_number           int64
win_fair_price        float64
win_starting_price    float64
winner                  int64
dtype: object

In [6]:
len(df)

86648